<center><span style="color:blue"><font size="6">Graph Visualization - Graph Notebook 활용편</font></span></center>  
  
<p><p>RDF 형태의 데이터는 주어-술어-목적어 이렇게 트리플 구조를 가지고 있기 때문에 이들을 연결하면 그래프 형태로 표현이 됩니다.<p>
이러한 특징으로 인해 그래프 형태로 visualization이 가능한데, 그래프로 그리기 위한 여러가지 라이브러리들이 존재합니다.<p>
그 중에서 간단하게 사용할 수 있는 graph-notebook 이라는 라이브러리가 있습니다.<p>
graph-notebook은 그래프 데이터베이스에 질의하여 결과를 가져올 수 있는데 <p>
SPARQL,Gremlin, Cypher 언어를 지원합니다. <p>
이 중에서 우리는 RDF를 사용하기 때문에 SPARQL언어를 사용하여 SPARQL Endpoint에 질의하여 결과를 그래프로 표현해보도록 하겠습니다.<p>
SPARQL은 로컬에 구성한 Endpoint로 질의를 하거나 다른 공개된 endpoint를 활용할 수 있습니다.<p>
    
 - 사용하는 라이브러리  
    graph-notebook  
    (https://pypi.org/project/graph-notebook) <p>
 - 필수적으로 설치가 되어 있어야 하는 라이브러리
    Tornado, RDFLib (보다 자세한 내용은 위 링크에서 확인하세요)

<div align="right">
    작성자 : 허홍수<br>
    e-mail : su4620@gmail.com<br>
    blog : http://joyhong.tistory.com<br>
</div>  

# 설정

제일 먼저 대상이 되는 endpoint 주소와 포트, 그리고 여러 정보를 설정합니다.  
대상 endpoint 주소는 (제 기준)  
http://localhost:3030/publicdata/query  입니다.  
수정이 필요한 곳 (host, port, sparql:path)

In [1]:
%%graph_notebook_config
{
    "host":"localhost",   
    "port":3030,          
    "auth_mode":"DEFAULT",
    "iam_credentials_provider_type":"ROLE",
    "load_from_s3_arn":"",
    "ssl": false,
    "aws_region":"us-east-1",
    "sparql": {
        "path": "publicdata/query"   
    }
}

set notebook config to:
{
  "host": "localhost",
  "port": 3030,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": "",
  "ssl": false,
  "aws_region": "us-east-1",
  "sparql": {
    "path": "publicdata/query"
  }
}


<b>쥬피터 화면을 넒게 쓰기 원하면 아래를 실행</b>

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 질의

In [2]:
%%sparql --expand-all 
select *
where{
    ?s ?p ?o.
} LIMIT 30




In [3]:
%%sparql --expand-all
prefix rs:    <http://localhost/resource/> 
select *
where{
    {
        filter(?s=<http://joyhong.tistory.com/resource/rg_220003>)
        ?s ?p ?o.
    } UNION {
        filter(?o=<http://joyhong.tistory.com/resource/rg_220003>)
        ?s ?p ?o.
    }
} LIMIT 30




# DBpedia 활용

## endpoint 설정

In [4]:
%%graph_notebook_config
{
    "host":"dbpedia.org",
    "port":443,
    "auth_mode":"DEFAULT",
    "iam_credentials_provider_type":"ROLE",
    "load_from_s3_arn":"",
    "ssl": true,
    "aws_region":"us-east-1"
}

set notebook config to:
{
  "host": "dbpedia.org",
  "port": 443,
  "auth_mode": "DEFAULT",
  "load_from_s3_arn": "",
  "ssl": true,
  "aws_region": "us-east-1",
  "sparql": {
    "path": "sparql"
  }
}


## 서울 출신 가수

In [5]:
%%sparql --expand-all
prefix dbo:    <http://dbpedia.org/ontology/> 
select ?s ?p ?o
where{
    ?s dct:subject <http://dbpedia.org/resource/Category:Singers_from_Seoul>.
    ?s ?p ?o .
    filter(?p!=rdf:type)
} LIMIT 500




## 아이유에 대한 정보

In [6]:
%%sparql --expand-all

SELECT * 
WHERE {
    ?s ?p ?o.
    FILTER(?s=<http://dbpedia.org/resource/IU_(singer)>)
}


## 기타

원하는 SPARQL을 작성하고 결과를 받아올 수는 있지만 그래프로 표현되지 않는 형식일 때는 그래프 탭이 나타나지 않는다.

In [7]:
%%sparql --expand-all

SELECT ?s ?label ?bdate (group_concat(?comt) as ?cmt) 
WHERE {
    ?s dct:subject dbc:Singers_from_Seoul.
    ?s rdfs:label ?label.
    FILTER(lang(?label)='ko')
    ?s dbo:birthDate ?bdate.
    ?s rdfs:comment ?comt.
} GROUP BY ?s ?label ?bdate
LIMIT 100



Finish